In [22]:
def HospitalDoctor():
    import arcpy   
    #creation of a list with the user's choices
    Choice=[]
    Choice.append(str(input("Do you want to consider the doctors in this analysis? (Yes, No) : ")))#user choice 
    arcpy.env.workspace = "Data Source" 
    #Verification of the existence of data   
    arcpy.Exists(arcpy.env.workspace) 
    #overwrite the output dataset
    arcpy.env.overwriteOutput = True
    #removes layers if they already exist
    intermediate_layers = ["buffer1","buffer2","PopMD","PopMH","PopNH","regions-20180101 selection.shp","PopNHFinal.shp"]
    for intermediate_layer in intermediate_layers:
        if arcpy.Exists(intermediate_layer):
            arcpy.Delete_management(intermediate_layer)
    #Buffer for hosiptals and doctors
    
    if Choice[0]!="Yes":
        #user's choice list
        MyList=[]
        MyList.append(str(input("Enter the buffer size for the hospitals (E.g. 20 Kilometers) : ")))
        MyList.append(float(input("Enter the coefficient relating to the minimum distance from a hospital (E.g. 3 ): ")))
        MyList.append(float(input("Enter the population density coefficient  (E.g. 6000 ): ")))
        MyList.append(str(input("Enter the population density category in the following list  (Ind, Ind_0_3,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p): ")))
        MyList.append(str(input("Enter the region you wish to select in the following list (Whole country,Corse,Île-de-France,Hauts-de-France,Nouvelle-Aquitaine,Normandie,Pays de la Loire,Centre-Val de Loire,Grand Est,PACA,Bretagne,Bourgogne-Franche-Comté,Occitanie,Auvergne-Rhône-Alpes): ")))
        
        print ("The program is running...")
        #calculation with the buffer, erase, near, addfield and calculatefield tools for the hospital
        arcpy.Buffer_analysis("Hospital/Hospital.shp", "buffer1",MyList[0])
        arcpy.Erase_analysis("Population density/Filosofi2015_carreaux_1000m_metropole.shp", "buffer1.shp", "PopNH")
        arcpy.Near_analysis("PopNH.shp", "Hospital/Hospital.shp")
        arcpy.AddField_management("PopNH.shp","Display","LONG")
        Expression=str(MyList[1])+"*!NEAR_DIST!+"+str(MyList[2])+"*!"+str(MyList[3])+"!"
        arcpy.CalculateField_management("PopNH.shp","Display",Expression,"PYTHON_9.3")
        #selection of the studied area 
        if MyList[4] != "Whole country":
            arcpy.MakeFeatureLayer_management("Region of France/regions-20180101.shp","Selecte")   
            qry="nom ="+"'"+str(MyList[4])+"'"
            arcpy.SelectLayerByAttribute_management("Selecte","NEW_SELECTION",qry)
            arcpy.CopyFeatures_management("Selecte", "regions-20180101 selection")
            arcpy.MakeFeatureLayer_management("PopNH.shp","Selecte2")
            arcpy.MakeFeatureLayer_management("regions-20180101 selection.shp","Selecte3")
            arcpy.SelectLayerByLocation_management ("Selecte2","INTERSECT","Selecte3")
            arcpy.CopyFeatures_management("Selecte2", "PopNHFinale.shp")
        else  :
            arcpy.CopyFeatures_management("PopNH.shp","PopNHFinale.shp")       
        print ("The program has finished running, open Arcgis Pro to see the results (layer PopNHFinale.shp)")
        
    else :
        #user's choice list
        MyList=[]
        MyList.append(str(input("Enter the buffer size for the hospitals (E.g. 20 Kilometers) : ")))
        MyList.append(str(input("Enter the buffer size for the doctors (E.g. 3 Kilometers) : ")))
        MyList.append(float(input("Enter the coefficient relating to the minimum distance from a hospital (E.g. 3 ): ")))
        MyList.append(float(input("Enter the coefficient relating to the minimum distance from a doctor (E.g. 0.5 ): ")))
        MyList.append(float(input("Enter the population density coefficient  (E.g. 6000 ): ")))
        MyList.append(str(input("Enter the population density category in the following list  (Ind, Ind_0_3,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p): ")))
        MyList.append(str(input("Enter the region you wish to select in the following list (Whole country,Corse,Île-de-France,Hauts-de-France,Nouvelle-Aquitaine,Normandie,Pays de la Loire,Centre-Val de Loire,Grand Est,PACA,Bretagne,Bourgogne-Franche-Comté,Occitanie,Auvergne-Rhône-Alpes): ")))
        
        print ("The program is running...")
        #calculation with the buffer, erase, near, addfield, calculatefield and deletefield tools for the hospital and doctor
        arcpy.Buffer_analysis("Hospital/Hospital.shp", "buffer1",MyList[0])
        arcpy.Buffer_analysis("Doctor/Doctor.shp", "buffer2",MyList[1])
        arcpy.Erase_analysis("Population density/Filosofi2015_carreaux_1000m_metropole.shp", "buffer1.shp", "PopMD")
        arcpy.Erase_analysis("PopMD.shp", "buffer2.shp", "PopMH")
        arcpy.Near_analysis("PopMH.shp", "Hospital/Hospital.shp")
        arcpy.Near_analysis("PopMH.shp", "Hospital/Hospital.shp") 
        arcpy.FeatureClassToFeatureClass_conversion("PopMH.shp", "Data Source", "PopNH.shp")
        arcpy.AddField_management("PopNH.shp","NEAR_FIDH","LONG")
        arcpy.CalculateField_management("PopNH.shp","NEAR_FIDH","!NEAR_FID!","PYTHON_9.3")
        arcpy.DeleteField_management("PopNH.shp","NEAR_FID")
        arcpy.AddField_management("PopNH.shp","NEAR_DISTH","LONG")
        arcpy.CalculateField_management("PopNH.shp","NEAR_DISTH","!NEAR_DIST!","PYTHON_9.3")
        arcpy.DeleteField_management("PopNH.shp","NEAR_DIST") 
        arcpy.Near_analysis("PopNH.shp", "Doctor/Doctor.shp")
        arcpy.AddField_management("PopNH.shp","Display","LONG")
        Expression=str(MyList[2])+"*!NEAR_DISTH!+"+str(MyList[3])+"*!NEAR_DIST!+"+str(MyList[4])+"*!"+str(MyList[5])+"!" 
        arcpy.CalculateField_management("PopNH.shp","Display",Expression,"PYTHON_9.3")
         #selection of the studied area 
        if MyList[6] != "Whole country":
            arcpy.MakeFeatureLayer_management("Region of France/regions-20180101.shp","Selecte")   
            qry="nom ="+"'"+str(MyList[6])+"'"
            arcpy.SelectLayerByAttribute_management("Selecte","NEW_SELECTION",qry)
            arcpy.CopyFeatures_management("Selecte", "regions-20180101 selection")
            arcpy.MakeFeatureLayer_management("PopNH.shp","Selecte2")
            arcpy.MakeFeatureLayer_management("regions-20180101 selection.shp","Selecte3")
            arcpy.SelectLayerByLocation_management ("Selecte2","INTERSECT","Selecte3")
            arcpy.CopyFeatures_management("Selecte2", "PopNHFinale.shp")
        else  :
            arcpy.CopyFeatures_management("PopNH.shp","PopNHFinale.shp")       
        print ("The program has finished running, open Arcgis Pro to see the results (layer PopNHFinale.shp)")
    

In [24]:
HospitalDoctor()

Do you want to consider the doctors in this analysis? (Yes, No) : No
Enter the buffer size for the hospitals (E.g. 20 Kilometers) : 10 Kilometers
Enter the coefficient relating to the minimum distance from a hospital (E.g. 3 ): 3
Enter the population density coefficient  (E.g. 6000 ): 500
Enter the population density category in the following list  (Ind, Ind_0_3,Ind_4_5,Ind_6_10,Ind_11_17,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p): Ind_25_39
Enter the region you wish to select in the following list (Whole country,Corse,Île-de-France,Hauts-de-France,Nouvelle-Aquitaine,Normandie,Pays de la Loire,Centre-Val de Loire,Grand Est,PACA,Bretagne,Bourgogne-Franche-Comté,Occitanie,Auvergne-Rhône-Alpes): PACA
The program is running...
The program has finished running, open Arcgis Pro to see the results (layer PopNHFinale.shp)
